# Chains

## ConversationChain

In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

In [3]:
chat = ChatOpenAI(model='gpt-4o-mini')
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input="Oi!")

C:\Users\francisco.procopio\AppData\Local\Temp\ipykernel_23240\2605409397.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\francisco.procopio\AppData\Local\Temp\ipykernel_23240\2605409397.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Oi!
AI:

> Finished chain.


"Oi! How's it going? What’s on your mind today?"

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Essa é uma conversa amigável entre um humano e uma IA.

    Conversa atual:

    {history}

    Human: {input}
    Ai:"""
)

In [5]:
chat = ChatOpenAI(model='gpt-4o-mini')
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input="Oi!")



> Entering new ConversationChain chain...
Prompt after formatting:

    Essa é uma conversa amigável entre um humano e uma IA.

    Conversa atual:

    

    Human: Oi!
    Ai:

> Finished chain.


'Oi! Como você está hoje? 😊'

In [6]:
chain.predict(input="Tudo bem! e você?")



> Entering new ConversationChain chain...
Prompt after formatting:

    Essa é uma conversa amigável entre um humano e uma IA.

    Conversa atual:

    Human: Oi!
AI: Oi! Como você está hoje? 😊

    Human: Tudo bem! e você?
    Ai:

> Finished chain.


'Estou bem, obrigado! Pronto para conversar e ajudar no que você precisar. O que você gostaria de fazer hoje? 😊'

## LLMChain

In [9]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

In [10]:
chat = ChatOpenAI(model='gpt-4o-mini')

In [21]:
prompt = PromptTemplate.from_template(
    """
    Qual o melhor nome para uma empresa que desenvolve o produto: {produto}
    Retorne apenas um nome.
    """
)

chain = LLMChain(
    llm=chat,
    prompt=prompt
)

In [22]:
produto = 'copos de vidro inquebráveis'

In [23]:
chain.invoke(produto)

{'produto': 'copos de vidro inquebráveis', 'text': 'VitraGuard'}

## SimpleSequentialChain

In [25]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(model='gpt-4o-mini')

In [26]:
prompt_nome = PromptTemplate.from_template(
    """
    Qual o melhor nome para uma empresa que desenvolve o produto: {produto}
    Retorne apenas um nome.
    """
)

chain_nome = LLMChain(
    llm=chat,
    prompt=prompt_nome
)

prompt_descricao = PromptTemplate.from_template(
    """
    Dado a empresa com o seguinte nome: {nome_empresa}.
    Dê uma descrição de até 50 palavras das atividades dessa empresa.
    """
)

chain_descricao = LLMChain(
    llm=chat,
    prompt=prompt_descricao
)

In [27]:
chain_final = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)

In [28]:
produto = 'Copos de vidro inquebráveis'

chain_final.invoke(produto)



> Entering new SimpleSequentialChain chain...
Vitraflex.
A Vitraflex é uma empresa especializada na fabricação e distribuição de produtos de vidro flexível, como embalagens, frascos e tarros. Com foco em inovação e qualidade, atende setores como cosméticos, alimentos e farmacêuticos, oferecendo soluções personalizadas que garantem durabilidade e sustentabilidade em suas práticas.

> Finished chain.


{'input': 'Copos de vidro inquebráveis',
 'output': 'A Vitraflex é uma empresa especializada na fabricação e distribuição de produtos de vidro flexível, como embalagens, frascos e tarros. Com foco em inovação e qualidade, atende setores como cosméticos, alimentos e farmacêuticos, oferecendo soluções personalizadas que garantem durabilidade e sustentabilidade em suas práticas.'}

## SequentialChain

In [29]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(model='gpt-4o-mini')

In [43]:
prompt_nome = PromptTemplate.from_template(
    """
    Qual o melhor nome para uma empresa que desenvolve o produto: {produto}
    Retorne apenas um nome.
    """
)

chain_nome = LLMChain(
    llm=chat,
    prompt=prompt_nome,
    output_key="nome_empresa"
)

prompt_descricao = PromptTemplate.from_template(
    """
    Dado a empresa com o seguinte nome: {nome_empresa} e que produz o seguinte produto: {produto}.
    Dê uma descrição de até 50 palavras das atividades dessa empresa.
    """
)

chain_descricao = LLMChain(
    llm=chat,
    prompt=prompt_descricao,
    output_key='descricao_empresa'
)

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em inglês para a empresa de nome: {nome_empresa} que possui a seguinte descrição: {descricao_empresa}.
    Retorne objetivamente apenas um nome para a empresa.
    """
)

chain_traducao = LLMChain(
    llm=chat,
    prompt=prompt_traducao,
    output_key='nome_ingles'
)

In [44]:
chain_final = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=['produto'],
    output_variables=['nome_empresa', 'descricao_empresa', 'nome_ingles'],
    verbose=True
)

In [45]:
produto = 'Copos de vidro inquebráveis'

resposta = chain_final.invoke(produto)



> Entering new SequentialChain chain...

> Finished chain.


In [46]:
resposta

{'produto': 'Copos de vidro inquebráveis',
 'nome_empresa': 'Vitrus.',
 'descricao_empresa': 'A Vitrus é especializada na produção de copos de vidro inquebráveis, combinando design moderno e tecnologia avançada para oferecer produtos resistentes e sustentáveis. Focada em qualidade e inovação, a empresa atende tanto o mercado doméstico quanto o corporativo, promovendo soluções práticas e elegantes para o cotidiano.',
 'nome_ingles': 'GlassGuard Innovations'}

In [47]:
resposta['produto']

'Copos de vidro inquebráveis'

In [48]:
resposta['nome_empresa']

'Vitrus.'

In [49]:
resposta['descricao_empresa']

'A Vitrus é especializada na produção de copos de vidro inquebráveis, combinando design moderno e tecnologia avançada para oferecer produtos resistentes e sustentáveis. Focada em qualidade e inovação, a empresa atende tanto o mercado doméstico quanto o corporativo, promovendo soluções práticas e elegantes para o cotidiano.'

In [50]:
resposta['nome_ingles']

'GlassGuard Innovations'